In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np

from PIL import Image
import os, glob, numpy as np
import pandas as pd
from sklearn import datasets, model_selection
from sklearn.model_selection import train_test_split


USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
import random
import numpy as np
import os
import cv2
import glob
from PIL import Image
import PIL.ImageOps    

#다음 변수를 수정하여 새로 만들 이미지 갯수를 정합니다.
num_augmented_images = (825-260)

file_path = './dataset/bad_train/'
file_names = os.listdir(file_path)
total_origin_image_num = len(file_names)
augment_cnt = 1

#for i in range(1, num_augmented_images):
    change_picture_index = random.randrange(1, total_origin_image_num-1)
    print(change_picture_index)
    print(file_names[change_picture_index])
    file_name = file_names[change_picture_index]
    
    origin_image_path = './dataset/bad_train/' + file_name
    image = Image.open(origin_image_path)
    random_augment = random.randrange(1,4)
    
    if(random_augment == 1):
        #이미지 좌우 반전
        print("invert")
        inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
        inverted_image.save(file_path + 'inverted_' + str(augment_cnt) + '.jpg')
        
    elif(random_augment == 2):
        #이미지 기울이기
        print("rotate")
        rotated_image = image.rotate(random.randrange(-20, 20))
        rotated_image.save(file_path + 'rotated_' + str(augment_cnt) + '.jpg')
        
    elif(random_augment == 3):
        #노이즈 추가하기
        img = cv2.imread(origin_image_path)
        print("noise")
        row,col,ch= img.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy_array = img + gauss
        noisy_image = Image.fromarray(np.uint8(noisy_array)).convert('RGB')
        noisy_image.save(file_path + 'noiseAdded_' + str(augment_cnt) + '.jpg')
        
    augment_cnt += 1

IndentationError: unexpected indent (<ipython-input-2-41b71315f118>, line 18)

In [ ]:
#noisy_image = Image.fromarray(np.uint8(noisy_array)).convert('RGB')

In [3]:
img_dir = './dataset'
categories = ['bad_train', 'good_train']
np_classes = len(categories)

X = []
y = []

for idx, cat in enumerate(categories):
    
    img_dir_detail = img_dir + "/" + cat
    files = glob.glob(img_dir_detail+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        try:
            img = Image.open(f)      #폴더를 열어주고 이미지를 읽음
            img = img.convert("RGB")   #RGB로 바꿈
            resize_img = img.resize((128,128)) #이미지 크기를 모두 학습시키기 쉽게 128x128크기
            
            r, g, b = resize_img.split() # 각 쪼갠 이미지를 255로 나눠서 0~1 사이의 값이 나오도록 정규화한다.
            r_resize_img = np.asarray(np.float32(r) / 255.0)
            b_resize_img = np.asarray(np.float32(g) / 255.0)
            g_resize_img = np.asarray(np.float32(b) / 255.0)
            
            rgb_resize_img = np.asarray([r_resize_img, b_resize_img, g_resize_img]) # 이렇게 가공한 이미지를 추가한다.
            
            X.append(rgb_resize_img)
            y.append(idx) # 라벨 (ant: 0, bee: 1)
            if i % 700 == 0: #700으로 하면 폴더 명과 파일 길이를 더 
                print(cat, " : ", f)
        except:
            print(cat, str(i)+" 번째에서 에러 ")


X = np.array(X, dtype='float32')
y = np.array(y, dtype='int64')
print(y)

#1 0  이면 bad
#0 1  이면 good

print("ok", len(y))

bad_train  파일 길이 :  824
bad_train  :  ./dataset/bad_train/inverted_496.jpg
bad_train  :  ./dataset/bad_train/b_200.jpg
good_train  파일 길이 :  825
good_train  :  ./dataset/good_train/g_418.jpg
good_train  :  ./dataset/good_train/g_198.jpg
[0 0 0 ... 1 1 1]
ok 1649


In [4]:
X_train = X
y_train = y

In [5]:
img_dir = './dataset'
categories = ['bad_test', 'good_test']
np_classes = len(categories)

X = []
y = []

for idx, cat in enumerate(categories):
    
    img_dir_detail = img_dir + "/" + cat
    files = glob.glob(img_dir_detail+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        try:
            img = Image.open(f)      #폴더를 열어주고 이미지를 읽음
            img = img.convert("RGB")   #RGB로 바꿈
            resize_img = img.resize((128,128)) #이미지 크기를 모두 학습시키기 쉽게 128x128크기
            
            r, g, b = resize_img.split() # 각 쪼갠 이미지를 255로 나눠서 0~1 사이의 값이 나오도록 정규화한다.
            r_resize_img = np.asarray(np.float32(r) / 255.0)
            b_resize_img = np.asarray(np.float32(g) / 255.0)
            g_resize_img = np.asarray(np.float32(b) / 255.0)
            
            rgb_resize_img = np.asarray([r_resize_img, b_resize_img, g_resize_img]) # 이렇게 가공한 이미지를 추가한다.
            
            X.append(rgb_resize_img)
            y.append(idx) # 라벨 (ant: 0, bee: 1)
            if i % 700 == 0: #700으로 하면 폴더 명과 파일 길이를 더 
                print(cat, " : ", f)
        except:
            print(cat, str(i)+" 번째에서 에러 ")


X = np.array(X, dtype='float32')
y = np.array(y, dtype='int64')


#1 0  이면 bad
#0 1  이면 good

print("ok", len(y))

bad_test  파일 길이 :  174
bad_test  :  ./dataset/bad_test/b_302.jpg
good_test  파일 길이 :  554
good_test  :  ./dataset/good_test/g_1271.jpg
ok 728


In [6]:
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.75, shuffle=True)
#stratify=target

In [7]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1649, 3, 128, 128)
(182, 3, 128, 128)
(546, 3, 128, 128)


In [8]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1649,)
(182,)
(546,)


In [9]:
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).long()
y_val = torch.from_numpy(y_val).long()
y_test = torch.from_numpy(y_test).long()

In [10]:
#TensorDataset을 DataLoader에 전달해서 데이터의 일부만 간단히 추출
train = TensorDataset(X_train, y_train)
val = TensorDataset(X_val, y_val)
test = TensorDataset(X_test, y_test)

#순서로 섞어서 32개씩 데이터를 변환하는 DataLoader 작성
train_loader = DataLoader(train, batch_size=32, shuffle=True, num_workers = 4)
val_loader = DataLoader(val, batch_size=32, shuffle=False, num_workers = 4)
test_loader = DataLoader(test, batch_size=32, shuffle=False, num_workers = 4)

In [11]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

1649
182
546


In [12]:
class subNet(nn.Module):
    def __init__(self):
        super(subNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5, stride=1, padding=(1, 1))
        self.conv2 = nn.Conv2d(32, 64, 5, stride=1, padding=(1, 1))
        self.conv3 = nn.Conv2d(64, 128, 5, stride=1, padding=(1, 1))
        self.conv4 = nn.Conv2d(128, 256, 5, stride=1,padding=(1, 1))
        
        self.pool = nn.MaxPool2d((2, 2), stride=2, dilation=(1, 1))
            
        self.fc1 = nn.Linear(256 * 29 * 29,2)
        self.fc2 = nn.Linear(512, 2)
            

            
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        #print(x.shape)
        
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        return x


model = subNet().to(DEVICE)

batch_size = 32

In [13]:
import torch
import torchvision
import torchsummary
model = subNet().to(DEVICE)
print(model)

subNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=(1, 1), ceil_mode=False)
  (fc1): Linear(in_features=215296, out_features=2, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [14]:
from torchsummary import summary
summary(model, input_size=(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 126, 126]           2,432
            Conv2d-2         [-1, 64, 124, 124]          51,264
         MaxPool2d-3           [-1, 64, 62, 62]               0
            Conv2d-4          [-1, 128, 60, 60]         204,928
            Conv2d-5          [-1, 256, 58, 58]         819,456
         MaxPool2d-6          [-1, 256, 29, 29]               0
            Linear-7                    [-1, 2]         430,594
Total params: 1,508,674
Trainable params: 1,508,674
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 24.99
Params size (MB): 5.76
Estimated Total Size (MB): 30.93
----------------------------------------------------------------


In [15]:
# https://hoya012.github.io/blog/DenseNet-Tutorial-2/
num_epoch = 500
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

total = 0
correct = 0
running_loss_history = []
running_correct_history = []
validation_running_loss_history = []
validation_running_correct_history = []


for epoch in range(num_epoch):  
    epoch = epoch+1
    total = 0
    running_correct = 0.0
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        X_train, y_train = data
        X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() #손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
       
        _, predicted = torch.max(outputs.data, 1)
        total += y_train.size(0)
        running_correct += (predicted == y_train).sum().item()
       # a = torch.tensor(torch.sum(predicted == y_train).item() / len(predicted))
    
    
    # validation part
    val_total = 0
    validation_running_loss = 0.0
    validation_running_correct = 0.0
    
    for i, data in enumerate(val_loader, 0):
        X_val, y_val = data
        X_val, y_val = X_val.to(DEVICE), y_val.to(DEVICE)
        outputs = model(X_val)
        
        val_loss = criterion(outputs, y_val) 
        validation_running_loss += val_loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        val_total += y_val.size(0)
        validation_running_correct += (predicted == y_val).sum().item()
        
        
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_correct / len(train_loader.dataset)
    running_loss_history.append(epoch_loss)
    running_correct_history.append(epoch_acc)

    val_epoch_loss = validation_running_loss / len(val_loader.dataset)
    val_epoch_acc = validation_running_correct / len(val_loader.dataset)
    validation_running_loss_history.append(val_epoch_loss)
    validation_running_correct_history.append(val_epoch_acc)
    
    print(epoch,"epoch")
    print("training loss: {:.5f}, acc: {:5f}".format(epoch_loss, epoch_acc*100))
    print("validation loss: {:.5f}, acc: {:5f}".format(val_epoch_loss, val_epoch_acc*100))
    
    print('---------------------------------------------------------------------') 

print('Finished Training')

1 epoch
training loss: 0.02307, acc: 64.099454
validation loss: 0.02396, acc: 39.560440
---------------------------------------------------------------------
2 epoch
training loss: 0.01491, acc: 75.500303
validation loss: 0.02554, acc: 65.934066
---------------------------------------------------------------------
3 epoch
training loss: 0.01419, acc: 79.563372
validation loss: 0.01165, acc: 88.461538
---------------------------------------------------------------------
4 epoch
training loss: 0.01135, acc: 83.687083
validation loss: 0.01204, acc: 89.560440
---------------------------------------------------------------------
5 epoch
training loss: 0.01152, acc: 82.898727
validation loss: 0.01525, acc: 80.769231
---------------------------------------------------------------------
6 epoch
training loss: 0.01177, acc: 82.656155
validation loss: 0.01351, acc: 83.516484
---------------------------------------------------------------------
7 epoch
training loss: 0.01132, acc: 84.354154
valid

53 epoch
training loss: 0.00001, acc: 100.000000
validation loss: 0.04279, acc: 85.714286
---------------------------------------------------------------------
54 epoch
training loss: 0.00001, acc: 100.000000
validation loss: 0.04330, acc: 85.714286
---------------------------------------------------------------------
55 epoch
training loss: 0.00001, acc: 100.000000
validation loss: 0.04378, acc: 85.714286
---------------------------------------------------------------------
56 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.04433, acc: 85.714286
---------------------------------------------------------------------
57 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.04476, acc: 85.714286
---------------------------------------------------------------------
58 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.04506, acc: 85.714286
---------------------------------------------------------------------
59 epoch
training loss: 0.00000, acc: 10

105 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06279, acc: 86.263736
---------------------------------------------------------------------
106 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06307, acc: 86.263736
---------------------------------------------------------------------
107 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06356, acc: 85.714286
---------------------------------------------------------------------
108 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06374, acc: 85.714286
---------------------------------------------------------------------
109 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06401, acc: 85.714286
---------------------------------------------------------------------
110 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.06427, acc: 86.263736
---------------------------------------------------------------------
111 epoch
training loss: 0.00000, 

156 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07683, acc: 85.714286
---------------------------------------------------------------------
157 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07731, acc: 85.714286
---------------------------------------------------------------------
158 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07757, acc: 85.714286
---------------------------------------------------------------------
159 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07764, acc: 85.714286
---------------------------------------------------------------------
160 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07819, acc: 85.714286
---------------------------------------------------------------------
161 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.07801, acc: 85.714286
---------------------------------------------------------------------
162 epoch
training loss: 0.00000, 

207 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.08929, acc: 86.263736
---------------------------------------------------------------------
208 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.08968, acc: 85.714286
---------------------------------------------------------------------
209 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.08982, acc: 86.813187
---------------------------------------------------------------------
210 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.09008, acc: 86.263736
---------------------------------------------------------------------
211 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.09016, acc: 86.263736
---------------------------------------------------------------------
212 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.09045, acc: 85.714286
---------------------------------------------------------------------
213 epoch
training loss: 0.00000, 

258 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10052, acc: 86.813187
---------------------------------------------------------------------
259 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10060, acc: 86.813187
---------------------------------------------------------------------
260 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10098, acc: 86.263736
---------------------------------------------------------------------
261 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10102, acc: 86.813187
---------------------------------------------------------------------
262 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10156, acc: 86.263736
---------------------------------------------------------------------
263 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.10148, acc: 86.813187
---------------------------------------------------------------------
264 epoch
training loss: 0.00000, 

309 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11097, acc: 86.813187
---------------------------------------------------------------------
310 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11115, acc: 86.813187
---------------------------------------------------------------------
311 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11130, acc: 86.813187
---------------------------------------------------------------------
312 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11147, acc: 86.813187
---------------------------------------------------------------------
313 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11192, acc: 86.813187
---------------------------------------------------------------------
314 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.11227, acc: 86.813187
---------------------------------------------------------------------
315 epoch
training loss: 0.00000, 

360 epoch
training loss: 0.00000, acc: 100.000000
validation loss: 0.13636, acc: 86.263736
---------------------------------------------------------------------
361 epoch
training loss: 0.06948, acc: 82.656155
validation loss: 0.02291, acc: 75.824176
---------------------------------------------------------------------
362 epoch
training loss: 0.01932, acc: 61.673742
validation loss: 0.02008, acc: 76.923077
---------------------------------------------------------------------
363 epoch
training loss: 0.01928, acc: 61.613099
validation loss: 0.02250, acc: 76.923077
---------------------------------------------------------------------
364 epoch
training loss: 0.01926, acc: 61.188599
validation loss: 0.02014, acc: 76.923077
---------------------------------------------------------------------
365 epoch
training loss: 0.01938, acc: 62.462098
validation loss: 0.02060, acc: 76.923077
---------------------------------------------------------------------
366 epoch
training loss: 0.01937, acc: 

412 epoch
training loss: 0.01949, acc: 60.097029
validation loss: 0.02105, acc: 76.923077
---------------------------------------------------------------------
413 epoch
training loss: 0.01944, acc: 60.097029
validation loss: 0.02105, acc: 76.923077
---------------------------------------------------------------------
414 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02103, acc: 76.923077
---------------------------------------------------------------------
415 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02106, acc: 76.923077
---------------------------------------------------------------------
416 epoch
training loss: 0.01950, acc: 60.097029
validation loss: 0.02101, acc: 76.923077
---------------------------------------------------------------------
417 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02103, acc: 76.923077
---------------------------------------------------------------------
418 epoch
training loss: 0.01949, acc: 6

464 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02106, acc: 76.923077
---------------------------------------------------------------------
465 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02104, acc: 76.923077
---------------------------------------------------------------------
466 epoch
training loss: 0.01945, acc: 60.097029
validation loss: 0.02108, acc: 76.923077
---------------------------------------------------------------------
467 epoch
training loss: 0.01950, acc: 60.097029
validation loss: 0.02105, acc: 76.923077
---------------------------------------------------------------------
468 epoch
training loss: 0.01948, acc: 60.097029
validation loss: 0.02108, acc: 76.923077
---------------------------------------------------------------------
469 epoch
training loss: 0.01947, acc: 60.097029
validation loss: 0.02106, acc: 76.923077
---------------------------------------------------------------------
470 epoch
training loss: 0.01948, acc: 6

In [ ]:
import matplotlib.pyplot as plt
def plot_acc(train_losses, valid_losses):
    '''
    training과 validation loss를 시각화하는 함수
    '''
    
    # plot style을 seaborn으로 설정
    plt.style.use('seaborn')
    
    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training accuracy') 
    ax.plot(valid_losses, color='red', label='Validation accuracy')
    ax.set(title="Accuracy over epochs", 
            xlabel='Epoch',
            ylabel='Accuracy') 
    ax.legend()
    fig.show()
    
    # plot style을 기본값으로 설정
    plt.style.use('default')

In [ ]:
def plot_loss(train_losses, valid_losses):
    '''
    training과 validation loss를 시각화하는 함수
    '''
    
    # plot style을 seaborn으로 설정
    plt.style.use('seaborn')

    train_losses = list(np.array(train_losses))
    valid_losses = list(np.array(valid_losses))

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # plot style을 기본값으로 설정
    plt.style.use('default')

In [ ]:
plot_acc(running_correct_history, validation_running_correct_history)

In [ ]:
plot_loss(running_loss_history, validation_running_loss_history)

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for epoch in range(num_epoch): 

    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            X_test, y_test = data
            X_test, y_test = X_test.to(DEVICE), y_test.to(DEVICE)
            outputs = model(X_test)
            
            
            test_loss += F.cross_entropy(outputs, y_test, reduction='sum').item()
            
            _, predicted = torch.max(outputs, 1)
            c = (predicted == y_test).squeeze()
            
            for i in range(y_test.shape[0]):
                label = y_test[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            
                total += y_test.size(0)
                correct += (predicted == y_test).sum().item()
                
            
        test_loss /= len(test_loader.dataset)
        print('[%d epoch] Test Accuracy: %d %%' % (epoch + 1, 100 * correct / total),', Loss:{:.4f}'.format(test_loss))
        print("Test Accuracy Accuracy : {:.5f}".format( 100 * correct / total))
                      

   
print('Accuracy of the network on the 1813 test images: %d %%' % (
    100 * correct / total)) 

for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        categories[i], 100 * class_correct[i] / class_total[i])) 


In [ ]:
from ignite.metrics import Accuracy, Precision, Recall

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y

binary_accuracy = Accuracy(thresholded_output_transform)
precision = Precision(thresholded_output_transform)
recall = Recall(thresholded_output_transform)

In [ ]:
def CalculateMetrics(model):
    with torch.no_grad():
        for images, labels in test_loader:
            if USE_CUDA:
                images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            binary_accuracy.update((predicted, labels))
            precision.update((predicted, labels))
            recall.update((predicted, labels))
            
        print('Model accuracy : ', binary_accuracy.compute())
        print('Model Precision : ', precision.compute().item())
        print('Model Recall : ', recall.compute().item())

In [ ]:
CalculateMetrics(model)

In [ ]:
def CalculateMetrics(model):
    with torch.no_grad():
        for images, labels in test_loader:
            if USE_CUDA:
                images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            binary_accuracy.update((predicted, labels))
            precision.update((predicted, labels))
            recall.update((predicted, labels))
            
        print('Model accuracy : ', binary_accuracy.compute())
        print('Model Precision : ', precision.compute().item())
        print('Model Recall : ', recall.compute().item())

In [ ]:
F1 = (precision * recall * 2 / (precision + recall)).mean()